In [1]:
# 주피터 노트북 파일의 병합
# 1. 먼저 병합 하려는 노트북 파일과 첨부 프로그램을 다운받아
#    별도의 폴더를 만들어 한 곳에 모읍니다. 
#    파일이름 순으로 병합되므로 파일 이름을 잘 조정해 놓습니다.원본 파일은 변경되지 않습니다.
# 2. 프로그램을 run하면 병합 파일이 생성됩니다. 병합된 결과 파일명은 "merged.ipynb" 입니다.

import json
import os

def get_files():
    # 병합할 노트북 파일의 list 만들기
    notebooks_to_merge = [file.name for file in os.scandir() if file.name.endswith('.ipynb') and file.is_file()]

    # 병합하는 노트북 파일을 정렬한다. 순서대로 병합하는 것으로 간주
    notebooks_to_merge.sort()

    print("다음 노트북이 병합됩니다.")
    for notebook in notebooks_to_merge:
        print(notebook)
    
    return notebooks_to_merge

def merge_notebooks(notebooks, merged_notebook):
    
    '''
    parameters:
    notebooks : 병합할 주피터노트북 파일들 이름
    merged_notebook : 병합된 결과물 파일
    returns : 없음
    '''
    with open (notebooks[0], mode = 'r', encoding = 'utf-8') as f:
        base = json.load(f)
    
    for notebook in notebooks[1:]:
        with open (notebook, mode = 'r', encoding = 'utf-8') as f:
            add = json.load(f)
            base['cells'].extend (add['cells']) 
     
    with open(merged_notebook, mode='w', encoding='utf-8') as f:
        json.dump(base, f)
    
    print(f'병합된 파일: {merged_notebook}')
    
notebooks_to_merge = get_files()
nfile = os.getcwd().split('\\')[-1]
merge_notebooks(notebooks_to_merge, f"{nfile}.ipynb")

다음 노트북이 병합됩니다.
01_updating_conjugate_priors.ipynb
02_pymc3_workflow.ipynb
03_bayesian_sharpe_ratio.ipynb
04_rolling_regression.ipynb
05_stochastic_volatility.ipynb
c.ipynb
병합된 파일: 10_bayesian_machine_learning.ipynb


In [29]:
import os
import json
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain import PromptTemplate

from dotenv import load_dotenv
load_dotenv()

# OpenAI API 키 설정

filepath = './10_bayesian_machine_learning.ipynb'
outpath = './10_bayesian_machine_learning_jeong.ipynb'


prompt_template = PromptTemplate(
    template="Translate the following text from English to Korean: {line}",
    input_variables=["line"]
)

# ChatOpenAI 인스턴스 생성
chat_openai = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'), model="gpt-3.5-turbo")

# .ipynb 파일 읽기
with open(filepath, 'r', encoding='utf-8') as f:
    notebook = json.load(f)

# 각 셀의 내용 번역
for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
        # 코드 셀은 건너뛰기
        continue

    source = cell['source']
    if isinstance(source, list):
        # 여러 줄인 경우
        translated = []
        for line in source:
            prompt = [HumanMessage(prompt_template.format(line=line))]
            response = chat_openai(prompt)
            translated.append(response.content.strip())
    else:
        # 한 줄인 경우
        prompt = [HumanMessage(prompt_template.format(line=source))]
        response = chat_openai(prompt)
        translated = [response.content.strip()]

    cell['source'] = translated

# 번역된 내용으로 .ipynb 파일 저장
with open(outpath, 'w', encoding='utf-8') as f:
    json.dump(notebook, f, ensure_ascii=False, indent=4)

print(f'{outpath} 번역 완료')

./10_bayesian_machine_learning_jeong.ipynb 번역 완료
